In [ ]:
import fintech_utils as ftu
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import norm, t

To install fintech_utils, go to fintech_utils_package in this repo and run `pip install -e .`

## 8.4

In [21]:
df = pd.read_csv("test7_1.csv")
df

,x1
0,0.071476
1,0.042703
2,0.046470
3,0.078857
4,0.071511
...,...
95,0.025379
96,0.006113
97,-0.020492
98,0.007548


In [22]:
def es_normal(x, alpha=0.05):
    mu = np.mean(x)
    sigma = np.std(x, ddof=1)
    z_alpha = norm.ppf(1 - alpha)
    phi = norm.pdf(z_alpha)
    es_abs = mu - sigma * phi / alpha
    es_diff = - sigma * phi / alpha
    return -es_abs, -es_diff
es_abs, es_diff = es_normal(df['x1'], alpha=0.05)
es_abs, es_diff

(0.05046784402005223, 0.09649358047292049)

In [23]:
ans = pd.read_csv("testout8_4.csv")
ans

,ES Absolute,ES Diff from Mean
0,0.050468,0.096494


## 8.5

In [24]:
df = pd.read_csv("test7_2.csv")
df

,x1
0,0.062695
1,-0.001343
2,0.058816
3,0.074756
4,0.014312
...,...
95,0.083073
96,0.125152
97,0.046132
98,0.036900


In [25]:
def es_t(x, alpha=0.05):
    nu_hat, loc_hat, scale_hat = t.fit(x)
    t_alpha = t.ppf(alpha, df=nu_hat)
    phi = t.pdf(t_alpha, df=nu_hat)
    es_abs = loc_hat - scale_hat * (nu_hat + t_alpha**2) / (nu_hat - 1) * phi / alpha
    es_diff = - scale_hat * (nu_hat + t_alpha**2) / (nu_hat - 1) * phi / alpha
    return -es_abs, -es_diff
es_t_abs, es_t_diff = es_t(df['x1'], alpha=0.05)
es_t_abs, es_t_diff


(0.07523208732199774, 0.12117246736935187)

In [26]:
ans = pd.read_csv("testout8_5.csv")
ans

,ES Absolute,ES Diff from Mean
0,0.075232,0.121172


## 8.6

In [27]:
df = pd.read_csv("test7_2.csv")
df

,x1
0,0.062695
1,-0.001343
2,0.058816
3,0.074756
4,0.014312
...,...
95,0.083073
96,0.125152
97,0.046132
98,0.036900


In [28]:
np.random.seed(123)
def es_simulation_t(x, alpha=0.05, n_simu=100_000):
    # Fit t distribution parameters to sample
    nu_hat, loc_hat, scale_hat = t.fit(x)
    
    # Simulate n_simu draws from fitted t
    simu = t.rvs(df=nu_hat, loc=loc_hat, scale=scale_hat, size=n_simu)
    
    # Compute empirical VaR and ES from simulated vector
    var_alpha = np.quantile(simu, alpha)
    es_abs = simu[simu <= var_alpha].mean()
    
    # ES deviation from mean
    es_diff = es_abs - loc_hat
    return es_abs, es_diff
es_simu_t_abs, es_simu_t_diff = es_simulation_t(df['x1'], alpha=0.05, n_simu=100000)
es_simu_t_abs, es_simu_t_diff

(-0.0741801355839984, -0.12012051563135254)

In [29]:
ans = pd.read_csv("testout8_6.csv")
ans

,ES Absolute,ES Diff from Mean
0,0.076906,0.122426


## 9.1 VaR/ES on 2 levels from simulated values - Copula

In [30]:
portfolio = pd.read_csv("test9_1_portfolio.csv")
returns = pd.read_csv("test9_1_returns.csv")
portfolio, returns

(  Stock  Holding  Starting Price Distribution
 0     A      100              20       Normal
 1     B      100              30            T,
             A         B
 0   -0.008018  0.029859
 1   -0.043707 -0.039298
 2   -0.008501 -0.025579
 3   -0.004756 -0.009689
 4    0.003788 -0.006877
 ..        ...       ...
 195  0.040275  0.009365
 196 -0.028035 -0.004859
 197  0.007395  0.015389
 198 -0.034845 -0.020320
 199  0.037054  0.030647
 
 [200 rows x 2 columns])

In [31]:
def var_normal(x, alpha=0.05):
    mu = np.mean(x)
    sigma = np.std(x, ddof=1)
    z_alpha = stats.norm.ppf(alpha)  
    var_signed = mu + sigma * z_alpha  
    var_abs = -var_signed              
    var_diff = -sigma * z_alpha        
    return var_abs, var_diff

def var_t(x, alpha=0.05):
    nu_hat, loc_hat, scale_hat = stats.t.fit(x)
    t_alpha = stats.t.ppf(alpha, df=nu_hat) 
    q_alpha = loc_hat + scale_hat * t_alpha  
    var_abs = -q_alpha                       
    var_diff = -scale_hat * t_alpha          
    return var_abs, var_diff

In [32]:
# from function in last week
var_a, _ = var_normal(returns.A.values, alpha=0.05)
var_b, _= var_t(returns.B.values, alpha=0.05)
var_a = var_a*100*20
var_b = var_b*100*30
var_a, var_b

(94.1999870886495, 108.58791171472252)

In [33]:
# ES 95
es_a, _ = es_normal(returns.A.values, alpha=0.05)
es_b, _ = es_t(returns.B.values, alpha=0.05)
es_a = es_a*100*20
es_b = es_b*100*30

In [34]:
denom_a = 20*100
denom_b = 30*100
var_pct_a = var_a / denom_a
var_pct_b = var_b / denom_b
es_pct_a = es_a / denom_a
es_pct_b = es_b / denom_b
var_pct_a, var_pct_b, es_pct_a, es_pct_b

(0.047099993544324745,
 0.03619597057157418,
 0.05895294104662377,
 0.050756316535026644)

In [35]:
def es_empirical(x, alpha=0.05):
    q_alpha = np.quantile(x, alpha)
    tail = x[x <= q_alpha]
    es = -tail.mean()
    return es, q_alpha

In [36]:
def portfolio_var_es_copula(returns, dists, holdings, prices, alpha=0.05, n_simu=1_000_000):
    """
        Get portfolio VaR and ES using a copula approach.
        Inputs:
            returns: DataFrame of asset returns, each column is an asset
            dists: list of distribution names for each asset, e.g. ['norm', 't']
            holdings: list of number of shares held for each asset
            prices: list of prices per share for each asset
            alpha: significance level for VaR/ES
            n_simu: number of simulations to run
        Outputs:
            VaR and ES in dollar terms
    """
    assets = returns.columns
    n_assets = len(assets)
    v0 = holdings * prices
    V0 = sum(v0)
    weights = v0 / V0

    # Fit marginals
    params = []
    uni = []
    for i, asset in enumerate(assets):
        if dists[i] == 'norm':
            mu, sigma = np.mean(returns[asset]), np.std(returns[asset], ddof=1)
            params.append((mu, sigma))
            uni.append(norm.cdf((returns[asset] - mu) / sigma))
        elif dists[i] == 't':
            nu, loc, scale = t.fit(returns[asset], floc=np.mean(returns[asset]))
            params.append((nu, loc, scale))
            uni.append(t.cdf((returns[asset] - loc) / scale, df=nu))
        else:
            raise ValueError(f"Unsupported distribution: {dists[i]}")

    # Estimate correlation (Gaussian copula)
    uni = np.array(uni).T # (n_samples, n_assets)
    z = norm.ppf(np.array(uni))
    rho = np.corrcoef(z.T)
    # Simulate correlated normals
    Z = np.random.multivariate_normal(np.zeros(n_assets), rho, n_simu)
    U = norm.cdf(Z)
    # Transform back to each marginal
    R_sim = np.zeros_like(U)
    for i in range(n_assets):
        if dists[i] == 'norm':
            mu, sigma = params[i]
            R_sim[:,i] = norm.ppf(U[:,i], loc=mu, scale=sigma)
        elif dists[i] == 't':
            nu, loc, scale = params[i]
            R_sim[:,i] = t.ppf(U[:,i], df=nu, loc=loc, scale=scale)
    # Compute portfolio returns
    R_port = R_sim @ weights
    # Compute VaR / ES
    q_alpha = np.quantile(R_port, alpha)
    var_pct = -q_alpha
    es_pct = -R_port[R_port <= q_alpha].mean()
    var_dollar = V0 * var_pct
    es_dollar = V0 * es_pct
    return {'VaR_dollar': var_dollar, 'ES_dollar': es_dollar, 'VaR_pct': var_pct, 'ES_pct': es_pct}

In [37]:
total = portfolio_var_es_copula(returns, ['norm', 't'], np.array([20, 30]), np.array([100, 100]), alpha=0.05, n_simu=1_000_000)
total

{'VaR_dollar': 151.59757757194518,
 'ES_dollar': 198.43700791536315,
 'VaR_pct': 0.030319515514389037,
 'ES_pct': 0.03968740158307263}

In [38]:
results = {
    'A': {'VaR95': var_a, 'ES95': es_a, 'VaR_pct': var_pct_a, 'ES_pct': es_pct_a},
    'B': {'VaR95': var_b, 'ES95': es_b, 'VaR_pct': var_pct_b, 'ES_pct': es_pct_b},
    'Total': {
        'VaR95': total['VaR_dollar'],
        'ES95': total['ES_dollar'],
        'VaR_pct': total['VaR_pct'],
        'ES_pct': total['ES_pct']
    }
}

pd.DataFrame(results).T.reset_index()

,index,VaR95,ES95,VaR_pct,ES_pct
0,A,94.199987,117.905882,0.047100,0.058953
1,B,108.587912,152.268950,0.036196,0.050756
2,Total,151.597578,198.437008,0.030320,0.039687


In [39]:
ans = pd.read_csv("testout9_1.csv")
ans

,Stock,VaR95,ES95,VaR95_Pct,ES95_Pct
0,A,94.460376,118.289371,0.047230,0.059145
1,B,107.880427,151.218174,0.035960,0.050406
2,Total,152.565684,199.704532,0.030513,0.039941
